# (not done) Node Classification with Graph Neural Networks
> 공부

- toc:true
- branch: master
- badges: true
- comments: false
- author: 최서연
- categories: [GNN]

### Import

In [1]:
import os
import pandas as pd
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

2022-05-29 15:27:00.913333: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-05-29 15:27:00.913359: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


### 데이터 구성

In [68]:
ex_X = pd.DataFrame(np.random.choice([0,1],p=[3/4,1/4],size=(1000,6)),columns = ['X1','X2','X3','X4','X5','X6'])
ex_X.shape

(1000, 6)

id의 class가 뭔지 구분할 term들 6개

In [69]:
ex_id = pd.DataFrame({'id' : np.array(range(1,1001))})
ex_id.shape

(1000, 1)

id

In [70]:
ex_Y = pd.DataFrame(np.random.randint(1,3,size=(1000,1)))
ex_Y.columns = ['name']
ex_Y['subject'] = ex_Y['name'].apply(lambda x : 'Deep learning' if x == 1 else 'Reinforcement learning' )
ex_Y.shape

(1000, 2)

class = 2개

In [72]:
papers = pd.concat([ex_id,ex_X,ex_Y['subject']],axis=1)
papers

,id,X1,X2,X3,X4,X5,X6,subject
0,1,1,0,0,0,0,0,Deep learning
1,2,0,0,0,0,0,0,Deep learning
2,3,0,1,1,0,0,1,Deep learning
3,4,0,0,1,1,0,0,Deep learning
4,5,0,0,0,0,0,1,Deep learning
...,...,...,...,...,...,...,...,...
995,996,0,1,0,0,0,0,Deep learning
996,997,0,1,0,1,0,1,Reinforcement learning
997,998,0,1,0,0,1,0,Deep learning
998,999,0,0,0,0,0,1,Reinforcement learning


### 그래프 표현

### ..

### 데이터 구성

### 데이터 구성